# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using NIfTI
using Plots

### Load data ###

In [23]:
# load data
ni = niread("data/nifti_dt.nii.gz") * 1e9;

112×112×50×1×6 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0 

In [25]:
x = 49
y = 60
z = 25

ni[x,y,z,1,:]  # gets xx, yy, zz, xy, xz, yz

6-element Vector{Float64}:
  1.4564411898732033
  0.10000896638606704
  1.8165984272400237
 -0.10748473638910738
 -0.16267717728446485
  1.443322461547325

### Construct manifold ###

In [20]:
d1, d2, d3, _, d = size(ni)
M_base = SymmetricPositiveDefinite(3)
M = PowerManifold(M_base, NestedPowerRepresentation(), d1, d2, d3)

PowerManifold(SymmetricPositiveDefinite(3), NestedPowerRepresentation(), 112, 112, 50)

In [26]:
function array2mat(array)
    result = zeros(3,3)
    result[1,1] = array[1]
    result[2,2] = array[2]
    result[3,3] = array[3]

    result[1,2] = array[4]
    result[1,2] = array[4]

    result[1,3] = array[5]
    result[1,3] = array[5]

    result[2,3] = array[6]
    result[2,3] = array[6]

    return result
end

array2mat (generic function with 1 method)

In [31]:
# construct data as points on the manifold
predata = [array2mat(ni[i,j,k,1,:]) for i=1:d1, j=1:d2, k=1:d3];

In [34]:
print(size(predata))

(112, 112, 50)

In [ ]:
# pick a 2D slice 
# xlims [34,81] y: [32,79] z:[8,30]